In [1]:
from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer
from module_utils import AdapterT5Block
import torch.nn as nn

In [2]:
import sys
sys.path.append("./hf_transformers/")

In [3]:
model_directory='./test'
batch_size=16

In [4]:
# down_projection = Linear(100, 30)
# activation = ReLU()
# up_projection = Linear(30, 100)


In [5]:
# class AdapterBlock(nn.Module):
#     def __init__(self, in_channels, hidden_dimension):
#         super().__init__()
#         self.norm = nn.LayerNorm(in_channels)
#         self.down_projection = Linear(in_channels, hidden_dimension)
#         self.activation = ReLU()
#         self.up_projection = Linear(hidden_dimension, in_channels)
        
        
#     def forward(self, x):
#         hidden = self.norm(x)
#         hidden = self.down_projection(hidden)
#         hidden = self.activation(hidden)
#         output = self.up_projection(hidden)
#         return output + x

        

In [6]:
base_model = f'./storage/checkpoint-37375'
tokenizer = T5Tokenizer.from_pretrained(base_model)
model = T5ForConditionalGeneration.from_pretrained(base_model)
model.resize_token_embeddings(len(tokenizer))
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(32104, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32104, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [7]:
model.config.adapter_size = 50
model.config.adapter_size

50

In [8]:
# from transformers.models.t5.modeling_t5 import T5Block
# class AdapterT5Block(T5Block):
#     def __init__(self, config, has_relative_attention_bias=False):
#         super().__init__(config, has_relative_attention_bias)
#         self.adapter = AdapterBlock(config.d_model, 30)
        
#     def forward(self, *args, **kwargs):
#         print('fkkkk')
#         output = super().forward(*args, **kwargs)
#         output_projected = self.adapter(output[0])
#         return (output_projected,) + output[1:]


In [9]:
import copy
encoder_config = copy.deepcopy(model.config)
encoder_config.is_decoder = False
encoder_config.use_cache = False
encoder_config.is_encoder_decoder = False
encoder_blocks = nn.ModuleList(
            [AdapterT5Block(encoder_config, has_relative_attention_bias=bool(i == 0)) for i in range(model.config.num_layers)]
        )

In [10]:
decoder_config = copy.deepcopy(model.config)
decoder_config.is_decoder = True
decoder_config.is_encoder_decoder = False
decoder_config.num_layers = model.config.num_decoder_layers
decoder_blocks = nn.ModuleList(
            [AdapterT5Block(decoder_config, has_relative_attention_bias=bool(i == 0)) for i in range(model.config.num_layers)]
        )

In [11]:
for i in range(len(encoder_blocks)):
    encoder_blocks[i].load_state_dict(model.encoder.block[i].state_dict(), strict=False)
    model.encoder.block[i] = encoder_blocks[i].to('cuda')    

In [12]:
for i in range(len(decoder_blocks)):
    decoder_blocks[i].load_state_dict(model.decoder.block[i].state_dict(), strict=False)
    model.decoder.block[i] = decoder_blocks[i].to('cuda')    

In [13]:
tokens = tokenizer(['hi how are you', 'thanks Im fine'], padding=True, return_tensors='pt').to('cuda')
tokens

{'input_ids': tensor([[7102,  149,   33,   25,    1],
        [2049, 1318, 1399,    1,    0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0]], device='cuda:0')}

In [14]:
import torch

from data_reader import GetDataAsPython
from prepare_data import create_data
from prepare_data import create_dataset
from prepare_data import extract_warning_types
from utils import boolean_string
from utils import get_current_time
import csv
import numpy as np
import random

from datetime import datetime
import argparse
import os

local = True

if local:
    storage_directory = './storage/'
    base_model = f'./{storage_directory}/checkpoint-37375'
    batch_size = 16
    codebert_address = "microsoft/codebert-base"
else:
    storage_directory = '/scratch/arminz/'
    batch_size = 64
    # base_model = f'/{storage_directory}/t5-small_global_repo-based_03-11-2021_15-28-40/checkpoint-37375/'
    base_model = f'{storage_directory}/checkpoint-37375'
    codebert_address = "/home/arminz/codebert-base"


repo = '/data/all/data/appium/appium' #,/data/all/data/request/request'#, args.repo
repo_percent = 1.0 # args.repo_percent

In [15]:

data = GetDataAsPython(f"{storage_directory}/data_and_models/data/data_autofix_tracking_repo_specific_final.json")
data_eslint = GetDataAsPython(f"{storage_directory}/data_and_models/data/data_autofix_tracking_eslint_final.json")
data += data_eslint
len(data)

104804

In [16]:
all_warning_types = extract_warning_types(data)

# In[16]:


(repo_train_inputs, repo_train_labels, repo_val_inputs, repo_val_labels, repo_test_inputs, repo_test_labels,
 repo_train_info, repo_val_info, repo_test_info,) = create_data(data, all_warning_types, include_warning=True,
                                                                design='repo-based-included', select_repo=repo)

# In[17]:


(general_train_inputs, general_train_labels, general_val_inputs, general_val_labels, general_test_inputs,
 general_test_labels, general_train_info, general_val_info, general_test_info,) = create_data(data, all_warning_types,
                                                                                              include_warning=True,
                                                                                              design='repo-based')

splitting by : repo-based-included
train size: 63
val size: 19
test size: 73
splitting by : repo-based
train size: 75087
val size: 25055
test size: 2383


In [17]:
samples = int(repo_percent * len(repo_train_inputs))
print(f'all repo samples: {len(repo_train_inputs)}\ntrain samples of repo: {samples} ({repo_percent})')
name='at'
exec_number = random.randint(0, 1000)


all repo samples: 63
train samples of repo: 63 (1.0)


In [18]:
tokenizer = T5Tokenizer.from_pretrained(base_model)

# In[48]:


train_dataset = create_dataset(repo_train_inputs[:samples], repo_train_labels[:samples], tokenizer, pad_truncate=True, max_length=128)
val_dataset = create_dataset(repo_val_inputs, repo_val_labels, tokenizer, pad_truncate=True)
# test_dataset = create_dataset(repo_val_inputs, repo_val_labels, tokenizer, pad_truncate=True)
# 
# In[49]:


now = datetime.now()
test_result_directory = f'{storage_directory}/fine-tune-result'
full_name = f'{name}_{exec_number}_{repo.rsplit("/", 1)[1][-20:]}_{repo_percent}_{samples}'
model_directory = f'{storage_directory}/tmp/{full_name}'
model_directory

/home/armin/TFix/env/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:185: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


'./storage//tmp/at_728_appium_1.0_63'

In [19]:
lr = 4e-3
ws = 300
wd = 0.4

In [20]:
for param in model.parameters():
    param.requires_grad = False
    
for block in model.encoder.block:
    for param in block.adapter.parameters():
        param.requires_grad = True
    
for block in model.decoder.block:
    for param in block.adapter.parameters():
        param.requires_grad = True
    

In [21]:
freezed, non_freezed = 0, 0
for item in model.parameters():
    if item.requires_grad:
        non_freezed += item.numel()
    else:
        freezed += item.numel()
print(non_freezed / (freezed + non_freezed))

0.010362426450774384


In [22]:
from transformers import EarlyStoppingCallback
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from transformers import T5Config
from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer
from transformers import set_seed

training_args = Seq2SeqTrainingArguments(
    output_dir=model_directory,
    num_train_epochs=70,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=ws,
    weight_decay=wd,
    logging_dir=model_directory,
    logging_steps=100,
    do_eval=True,
    evaluation_strategy="epoch",
    learning_rate=lr,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=1,
    eval_accumulation_steps=1,  # set this lower, if testing or validation crashes
    disable_tqdm=False,
    predict_with_generate=True,  # never set this to false.
    seed=42,  # default value
)


In [23]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    optimizers=[torch.optim.Adam(params=model.parameters(), lr=lr), None],
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
    #     compute_metrics=compute_metrics
)

In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,No log,0.074225,0.063200,300.730000
2,No log,0.074176,0.062500,303.889000
3,No log,0.074084,0.061100,310.995000
4,No log,0.073948,0.061300,310.075000
5,No log,0.073778,0.068200,278.737000
6,No log,0.073587,0.061700,308.054000
7,No log,0.073381,0.061500,308.788000
8,No log,0.073136,0.064600,294.145000
9,No log,0.072833,0.063400,299.895000
10,No log,0.072545,0.061900,306.857000


Some weights of the model checkpoint at ./storage//tmp/at_728_appium_1.0_63/checkpoint-60 were not used when initializing T5ForConditionalGeneration: ['encoder.block.0.adapter.norm.weight', 'encoder.block.0.adapter.norm.bias', 'encoder.block.0.adapter.down_projection.weight', 'encoder.block.0.adapter.down_projection.bias', 'encoder.block.0.adapter.up_projection.weight', 'encoder.block.0.adapter.up_projection.bias', 'encoder.block.1.adapter.norm.weight', 'encoder.block.1.adapter.norm.bias', 'encoder.block.1.adapter.down_projection.weight', 'encoder.block.1.adapter.down_projection.bias', 'encoder.block.1.adapter.up_projection.weight', 'encoder.block.1.adapter.up_projection.bias', 'encoder.block.2.adapter.norm.weight', 'encoder.block.2.adapter.norm.bias', 'encoder.block.2.adapter.down_projection.weight', 'encoder.block.2.adapter.down_projection.bias', 'encoder.block.2.adapter.up_projection.weight', 'encoder.block.2.adapter.up_projection.bias', 'encoder.block.3.adapter.norm.weight', 'encod

TrainOutput(global_step=76, training_loss=0.1280616835543984, metrics={'train_runtime': 17.1836, 'train_samples_per_second': 16.295, 'total_flos': 56194512611328, 'epoch': 19.0})

In [25]:
print('eval', trainer.evaluate()['eval_loss'])

eval 0.07373589277267456


In [26]:
# best_model_dir = f'./test'
# trainer.save_model(best_model_dir)
# print('best model dir', best_model_dir)

In [27]:
model.save_pretrained('./test')
tokenizer.save_pretrained('./test')

('./test/tokenizer_config.json',
 './test/special_tokens_map.json',
 './test/spiece.model',
 './test/added_tokens.json')

In [28]:
for i, block in enumerate(model.encoder.block):
    torch.save(block.adapter.state_dict(), f'./test/adapter-encoder-{i}')

In [29]:
for i, block in enumerate(model.decoder.block):
    torch.save(block.adapter.state_dict(), f'./test/adapter-decoder-{i}')